## Overview

**Problema 2:** Você deverá implementar um programa, para ler, tratar e particionar os dados.

O arquivo fonte está disponível em https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz

**Data Quality**
* Higienizar e homogenizar o formato da coluna document
* Detectar através da coluna document se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação
* Higienizar e homogenizar o formato da coluna birthDate
* Existem duas colunas nesse dataset que em alguns registros estão trocadas. Quais são essas colunas?
* Corrigir os dados com as colunas trocadas
* Além desses pontos, existem outras tratamentos para homogenizar esse dataset. Aplique todos que conseguir.

**Agregação dos dados**
* **Quais são as 5 PF que mais gastaram (totalSpent)?**

* Maristela | 38540.65
* Rute | 38232.41
* Hosana | 37581.27
* Joceli | 36718.52
* Edelswitha | 35825.10


* **Qual é o valor de gasto médio por estado (state)?**

state	Media_UF
* Tocantins	501.90156334817055
* São Paulo	501.4107574178689
* Sergipe	498.24009925154587
* Santa Catarina	499.5841437224666
* SP	496.2803490129176
* SC	502.8878848657253
* Roraima	498.87567542316947
* Rondônia	500.13911271113074
* Rio de Janeiro	502.2880546357611
* Rio Grande do Sul	499.5195930327202
* Rio Grande do Norte	499.03889100506063
* RS	501.6273259716892
* RR	502.7863956691963
* RO	496.52307617235135
* RN	500.395509025621
* RJ	502.57091764934245
* Piauí	500.4632056845298
* Pernambuco	501.64938355420344
* Pará	500.34804745023183
* Paraíba	497.07039478706525
* Paraná	497.57960315732157
* PR	503.36779245283043
* PI	497.8490320257356
* PE	503.0815605590065
* PB	501.0952403589623
* PA	498.2318289114318
* Minas Gerais	499.53209412922286
* Mato Grosso do Sul	496.5547328990226
* Mato Grosso	501.3442530297263
* Maranhão	499.1732760513894
* MT	499.30269113445974
* MS	502.13242892936324
* MA	501.5515811965812
* Goiás	498.568081146371
* GO	503.6919138339542
* Espírito Santo	504.5003697212026
* ES	499.1019225195629
* Distrito Federal	501.64061679505267
* DF	496.8208284776967
* Ceará	501.71070827872444
* CE	497.1764769282587
* Bahia	497.7505410447763
* BA	499.67140211640225
* Amazonas	499.5002347774225
* Amapá	502.8261998913633
* Alagoas	502.7766122404253
* Acre	503.31984563253013
* AP	505.5490040093874
* AM	498.1124351824675
* AL	498.5047410222678
* AC	501.7259968255494

* **Qual é o valor de gasto médio por jobArea?**
Resposta SQL no final

* **Qual é a PF que gastou menos (totalSpent)?**
Sr. Enio Souza | 0

* **Quantos nomes e documentos repetidos existem nesse dataset?**
Resposta SQL no final

* **Quantas linhas existem nesse dataset?**
1000000

**Particionamento de dados tratados com as regras descritas em DATA QUALITY**
* Particionar em arquivos PARQUET por estado (state)
* Particionar em arquivos CSV por ano/mes/dia de nascimento (birthDate)

In [0]:
%python 

import re
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

from pyspark.sql.types import BooleanType
from pyspark.sql import functions as F

from pyspark.sql.functions import regexp_replace
import pyspark.sql.functions as f
from pyspark.sql.functions import when


import pandas as pd
from datetime import datetime
from dateutil.parser import parse

In [0]:
# File location and type
file_location = "/FileStore/tables/people_v2_1E6.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent
76684148787,Charlleny Braga,Oficial Criativo Dinâmico,Configuração,Estrategista,(62) 4216-9799,20-05-1972,Município de Iara,Goiás,913.8
85704855733,Newton Saraiva,Administrador Comunicações Internacional,Prestação de contas,Facilitador,Aplicações,10-Jun-1982,Município de Neide do Sul,RR,57.26
15664328373377,Dr. Sr. Solange Macedo,Designer Identidade Direto,Métricas,null,+55 (95) 7143-3307,05/16/1968,Município de Santo,RO,660.71
02.328.238/0877-86,Celina Carvalho Jr.,null,Qualidade,Ligação,+55 (58) 4136-5577,"19810417,",Reis do Norte,RO,542.51
30073687408740,Aurilo Martins,Especialista Paradigma Internacional,Programa,Executivo,(96) 47498-7325,04/07/1980,Município de Aminadab,RO,104.33
77145788233,Sr. Sávio Albuquerque,Estrategista Branding Interno,Resposta,Planejador,+55 (06) 0512-5291,"19830618,",Município de Balmes,Sergipe,769.26
75752983045,Sr. Everaldo Saraiva Filho,Técnico Táticas Produtos,Marketing,Diretor,(33) 78784-4156,27-Oct-1980,Nova Olga do Sul,Minas Gerais,575.1
88363035521,Martina Moreira,Associado Contas Legado,Marketing,Designer,+55 (30) 1195-8461,09/08/1978,Reis de Nossa Senhora,BA,381.15
46182752785,Benedikt-Jan Pereira,Associado Contas Produtos,Contas,Gerente,+55 (36) 8023-5165,"19771120,",Melo do Sul,Pernambuco,496.97
151.113.814-90,Izamar Melo Filho,Designer Configuração Distrito,Mercados,Especialista,(37) 02302-7101,19-01-1987,null,Mato Grosso do Sul,400.18


In [0]:
# Create a view or table

temp_table_name = "people_v2_1E6_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `people_v2_1E6_csv`

document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent
76684148787,Charlleny Braga,Oficial Criativo Dinâmico,Configuração,Estrategista,(62) 4216-9799,20-05-1972,Município de Iara,Goiás,913.8
85704855733,Newton Saraiva,Administrador Comunicações Internacional,Prestação de contas,Facilitador,Aplicações,10-Jun-1982,Município de Neide do Sul,RR,57.26
15664328373377,Dr. Sr. Solange Macedo,Designer Identidade Direto,Métricas,null,+55 (95) 7143-3307,05/16/1968,Município de Santo,RO,660.71
02.328.238/0877-86,Celina Carvalho Jr.,null,Qualidade,Ligação,+55 (58) 4136-5577,"19810417,",Reis do Norte,RO,542.51
30073687408740,Aurilo Martins,Especialista Paradigma Internacional,Programa,Executivo,(96) 47498-7325,04/07/1980,Município de Aminadab,RO,104.33
77145788233,Sr. Sávio Albuquerque,Estrategista Branding Interno,Resposta,Planejador,+55 (06) 0512-5291,"19830618,",Município de Balmes,Sergipe,769.26
75752983045,Sr. Everaldo Saraiva Filho,Técnico Táticas Produtos,Marketing,Diretor,(33) 78784-4156,27-Oct-1980,Nova Olga do Sul,Minas Gerais,575.1
88363035521,Martina Moreira,Associado Contas Legado,Marketing,Designer,+55 (30) 1195-8461,09/08/1978,Reis de Nossa Senhora,BA,381.15
46182752785,Benedikt-Jan Pereira,Associado Contas Produtos,Contas,Gerente,+55 (36) 8023-5165,"19771120,",Melo do Sul,Pernambuco,496.97
151.113.814-90,Izamar Melo Filho,Designer Configuração Distrito,Mercados,Especialista,(37) 02302-7101,19-01-1987,null,Mato Grosso do Sul,400.18


In [0]:
%python


def validar_cpf(cpf):
       
    cpf = ''.join(re.findall('\d', str(cpf)))

    if (not cpf) or (len(cpf) < 11):
        return False

    # Pega apenas os 9 primeiros dígitos do CPF e gera os 2 dígitos que faltam
    inteiros = list(map(int, cpf))    
    
    novo = inteiros[:9]
    
    while len(novo) < 11:
        r = sum([(len(novo)+1-i)*v for i,v in enumerate(novo)]) % 11

        if r > 1:
            f = 11 - r
        else:
            f = 0
        novo.append(f)

        # Se o número gerado coincidir com o número original, é válido
        if novo == inteiros:
            return cpf    
    return False

validar_cpfUDF = udf(lambda z:validar_cpf(z),StringType())

In [0]:
%python


def validar_cnpj(cnpj):
      
    cnpj = ''.join(re.findall('\d', str(cnpj)))
     
    if (not cnpj) or (len(cnpj) < 14):
        return False

    # Pega apenas os 12 primeiros dígitos do CNPJ e gera os 2 dígitos que faltam
    inteiros = list(map(int, cnpj))
    novo = inteiros[:12]

    prod = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    while len(novo) < 14:
        r = sum([x*y for (x, y) in zip(novo, prod)]) % 11
        if r > 1:
            f = 11 - r
        else:
            f = 0
        novo.append(f)
        prod.insert(0, 6)

    # Se o número gerado coincidir com o número original, é válido
    if novo == inteiros:
        return cnpj
    return False  


validar_cnpjUDF = udf(lambda z:validar_cnpj(z),StringType()) 

In [0]:
%python

def tipo_pessoa(documento):
    if (validar_cpf(documento) != False):
        return "PF"
    elif (validar_cnpj(documento) != False):
        return "PJ"
      
tipo_pessoaUDF = udf(lambda z:tipo_pessoa(z),StringType())     

In [0]:
%python

def formata_cpf_cnpj(documento):
    
    if (validar_cpf(documento) != False):
        documento = validar_cpf(documento)
        if len(documento) < 11:
            documento = documento.zfill(11)
        return '{}.{}.{}-{}'.format(documento[:3], documento[3:6], documento[6:9], documento[9:])       
        
    elif (validar_cnpj(documento) != False):
        documento = validar_cnpj(documento)
        if len(documento) < 14:
            documento = documento.zfill(14)
        return '{}.{}.{}/{}-{}'.format(documento[:2], documento[2:5], documento[5:8], documento[8:12], documento[12:])     
    else:
        return "Documento inválido"
      
tipo_pessoaUDF = udf(lambda z:tipo_pessoa(z),StringType())   


formata_cpf_cnpjUDF = udf(lambda z:formata_cpf_cnpj(z),StringType())

In [0]:
%python

def is_valid_date(date):
    if date:
        try:
            parse(date)            
            return True
        except:
            return False
    return False
  

def formata_data(data):
    if (is_valid_date):
        data = pd.to_datetime(data)
        return data.strftime('%Y-%m-%d')    
    
formata_dataUDF = udf(lambda z:formata_data(z),StringType())



In [0]:
%python

def formata_estado(uf):
    if (uf == 'Acre'):
        return 'AC'
    
    elif (uf == 'Alagoas'):
        return 'AL'
    
    elif (uf == 'Amapá'):
        return 'AP'
    
    elif (uf == 'Amazonas'):
        return 'AM'      
    
    elif (uf == 'Bahia'):
        return 'BA'            
    
    elif (uf == 'Ceará'):
        return 'CE'      
      
    elif (uf == 'Distrito Federal'):
        return 'DF'      
    
    elif (uf == 'Espírito Santo'):
        return 'ES'            
    
    elif (uf == 'Goiás'):
        return 'GO'            
    
    elif (uf == 'Distrito Federal'):
        return 'DF'            
    
    elif (uf == 'Maranhão'):
        return 'MA'
    
    elif (uf == 'Mato Grosso'):
        return 'MT'
    
    elif (uf == 'Mato Grosso do Sul'):
        return 'MS'      
    
    elif (uf == 'Minas Gerais'):
        return 'MG'            
    
    elif (uf == 'Paraná'):
        return 'PR'      
      
    elif (uf == 'Paraíba'):
        return 'PB'      
    
    elif (uf == 'Pará'):
        return 'PA'            
    
    elif (uf == 'Pernambuco'):
        return 'PE'            
    
    elif (uf == 'Piauí'):
        return 'PI'   
    
    elif (uf == 'Rio Grande do Norte'):
        return 'RN'
    
    elif (uf == 'Rio Grande do Sul'):
        return 'RS'      
    
    elif (uf == 'Rio de Janeiro'):
        return 'RJ'            
    
    elif (uf == 'Rondônia'):
        return 'RO'      
      
    elif (uf == 'Roraima'):
        return 'RR'      
    
    elif (uf == 'Santa Catarina'):
        return 'SC'            
    
    elif (uf == 'Sergipe'):
        return 'SE'            
    
    elif (uf == 'São Paulo'):
        return 'SP'  

    elif (uf == 'Tocantins'):
        return 'TO'        
    
    else:
        return uf

formata_estadoUDF = udf(lambda z:formata_estado(z),StringType())

 

In [0]:
def num_there(s):
    return any(i.isdigit() for i in s)

num_thereUDF = udf(lambda z:num_there(z),StringType())      
 

In [0]:

dfVal = df.withColumn('documento_f', formata_cpf_cnpjUDF(df.document))\
          .withColumn('tipo_pessoa',tipo_pessoaUDF(df.document))\
          .withColumn('birthDate_f',formata_dataUDF(df.birthDate))\
          .withColumn('state_f',formata_estadoUDF(df.state))\
          .withColumn('jobArea1',
                      when(num_thereUDF(df.jobArea) == False, df.jobArea))\
          .withColumn('jobArea2',
                      when(num_thereUDF(df.phoneNumber) == False, df.phoneNumber))\
          .withColumn('phoneNumber1',
                      when(num_thereUDF(df.phoneNumber) == True, df.phoneNumber))\
          .withColumn('phoneNumber2',
                      when(num_thereUDF(df.jobArea) == True, df.jobArea))
                      

  
  
dfVal = dfVal.select(dfVal["documento_f"].alias('documento'),
             dfVal["tipo_pessoa"],
             dfVal["name"],
             dfVal["job"],
             dfVal["jobArea1"].alias('jobArea'),
             dfVal["jobArea2"],
             dfVal["jobType"],
             dfVal["phoneNumber1"].alias('phoneNumber'),
             dfVal["phoneNumber2"],
             dfVal["birthDate_f"].alias('birthDate'),
             dfVal["city"],
             dfVal["state_f"].alias('state'),
             dfVal["totalSpent"],
            )

dfVal.display()


documento,tipo_pessoa,name,job,jobArea,jobArea2,jobType,phoneNumber,phoneNumber2,birthDate,city,state,totalSpent
766.841.487-87,PF,Charlleny Braga,Oficial Criativo Dinâmico,Configuração,null,Estrategista,(62) 4216-9799,null,1972-05-20,Município de Iara,GO,913.8
857.048.557-33,PF,Newton Saraiva,Administrador Comunicações Internacional,Prestação de contas,Aplicações,Facilitador,null,null,1982-06-10,Município de Neide do Sul,RR,57.26
15.664.328/3733-77,PJ,Dr. Sr. Solange Macedo,Designer Identidade Direto,Métricas,null,null,+55 (95) 7143-3307,null,1968-05-16,Município de Santo,RO,660.71
02.328.238/0877-86,PJ,Celina Carvalho Jr.,null,Qualidade,null,Ligação,+55 (58) 4136-5577,null,1981-04-17,Reis do Norte,RO,542.51
30.073.687/4087-40,PJ,Aurilo Martins,Especialista Paradigma Internacional,Programa,null,Executivo,(96) 47498-7325,null,1980-04-07,Município de Aminadab,RO,104.33
771.457.882-33,PF,Sr. Sávio Albuquerque,Estrategista Branding Interno,Resposta,null,Planejador,+55 (06) 0512-5291,null,1983-06-18,Município de Balmes,SE,769.26
757.529.830-45,PF,Sr. Everaldo Saraiva Filho,Técnico Táticas Produtos,Marketing,null,Diretor,(33) 78784-4156,null,1980-10-27,Nova Olga do Sul,MG,575.1
883.630.355-21,PF,Martina Moreira,Associado Contas Legado,Marketing,null,Designer,+55 (30) 1195-8461,null,1978-09-08,Reis de Nossa Senhora,BA,381.15
461.827.527-85,PF,Benedikt-Jan Pereira,Associado Contas Produtos,Contas,null,Gerente,+55 (36) 8023-5165,null,1977-11-20,Melo do Sul,PE,496.97
151.113.814-90,PF,Izamar Melo Filho,Designer Configuração Distrito,Mercados,null,Especialista,(37) 02302-7101,null,1987-01-19,null,MS,400.18


In [0]:
#Salva arquivo particionado parquet
dfVal.write.partitionBy("state").parquet("adl://adlalianca.azuredatalakestore.net/rao/teste/problema2_state.parquet")

#Salva arquivo particionado csv
dfVal.write.partitionBy("birthDate").format("com.databricks.spark.csv").save("adl://adlalianca.azuredatalakestore.net/rao/teste/problema2_birthDate.csv")

In [0]:
# Create a view or table

temp_table_name = "people"

dfVal.createOrReplaceTempView(temp_table_name)

In [0]:
%sql
--Quais são as 5 PF que mais gastaram (totalSpent)?

select name, sum(totalSpent) from people
group by name
order by sum(totalSpent) desc
limit 5


name,sum(totalSpent)
Maristela,38540.65
Rute,38232.409999999996
Hosana,37581.270000000004
Joceli,36718.520000000004
Edelswitha,35825.1


In [0]:
%sql
--Qual é o valor de gasto médio por estado (state)?
select state, avg(totalSpent) as Media_UF from people
group by state
order by state desc

state,Media_UF
TO,501.90156334817055
SP,498.69447971508157
SE,498.24009925154587
SC,501.34561245212296
RS,500.6309834299485
RR,500.9563978397838
RO,498.2373178493516
RN,499.75601769911486
RJ,502.43856869884104
PR,500.6442003073772


In [0]:
%sql
--Qual é o valor de gasto médio por jobArea?
select JobArea, avg(totalSpent) as Media_JobArea from people
group by JobArea
order by JobArea desc

JobArea,Media_JobArea
Usabilidade,506.59700050998424
Táticas,498.41377476456597
Soluções,501.4326185234692
Segurança,503.2346697670781
Resposta,499.40945073523625
Rede,500.935349327915
Qualidade,501.1288917887188
Programa,502.6645129805982
Prestação de contas,498.82161848341275
Pesquisa,498.8122161229609


In [0]:
%sql
--Qual é a PF que gastou menos (totalSpent)?
select name, sum(totalSpent) from people
where tipo_pessoa == 'PF' and totalSpent is not null
group by name
order by sum(totalSpent)
limit 1


name,sum(totalSpent)
Sr. Enio Souza,0.0


In [0]:
%sql
-- Quantos nomes repetidos existem nesse dataset?
select count(x.qtde) as qtde from (select name, count(*) qtde from people
group by name
having count(*) > 1) x


qtde
173513


In [0]:
%sql
-- Quantos documentos repetidos existem nesse dataset?
select count(x.qtde) as qtde from (select documento, count(*) qtde from people
group by documento
having count(*) > 1) x

qtde
430


In [0]:
%python
#Quantas linhas existem nesse dataset?

dfVal.count()

Out[15]: 1000000